In [32]:
import numpy as np
import pandas as pd
import string
import re

In [33]:
data = pd.read_csv('/content/training_data.csv')
data.head()

,tweet,sarcasm,sentiment,dialect
0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa
1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa
2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa
3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf
4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa


In [34]:
data[:20]

,tweet,sarcasm,sentiment,dialect
0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa
1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa
2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa
3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf
4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa
5,"""طبيب #المقاصة: إصابة #أحمد_الشيخ بسيطة تحتاج ...",False,NEG,msa
6,مرسي : مش هنام اكتر من اربع ساعات في اليوم.......,True,NEG,egypt
7,#انتخبوا_البرص مشكلة,True,NEG,egypt
8,"""Retweeted هاشتاق العرب (@TheArabHash):#فيديو ...",False,POS,msa
9,▪️محافظ العاصمة تفقد سير العملية الانتخابية في...,False,NEU,msa


In [35]:
training_text = data['tweet']
training_text

0        "د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...
1        "مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...
2        “الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...
3        "@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...
4        "قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...
                               ...                        
12543    هلأ صاير انت يا #فلعوط بدك تعطي محاضرات ع تويت...
12544    "لا اله الا الله💜#أيفون_البروفيسور https://t.c...
12545    "RT @turkyepost: #أردوغان : إذا كان المرتكب غي...
12546    RT @Yousiif65: هاري بوتر؟👓🎩 https://t.co/959Oo...
12547          النت يلعن شكله طول الوقت تمام لين بدا الحفل
Name: tweet, Length: 12548, dtype: object

In [36]:
with open('/content/stopwords.txt','r') as sw:
    stop_words = sw.read().split('\n')


In [53]:
START_OF_LINE = r"^"
OPTIONAL = "?"
ANYTHING = "."
ZERO_OR_MORE = "*"
ONE_OR_MORE = "+"

SPACE = "\s"
SPACES = SPACE + ONE_OR_MORE
NOT_SPACE = "[^\s]" + ONE_OR_MORE
EVERYTHING_OR_NOTHING = ANYTHING + ZERO_OR_MORE

ERASE = ""
FORWARD_SLASH = "\/"
NEWLINES = r"[\r\n]"

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [64]:
def preprocess(text):
    
    punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
    print(str(text))
    #mentions
    text = re.sub('@[\w]+','', text)

    #RT
    RE_TWEET = "RT" + SPACES
    text = re.sub(RE_TWEET,'', text)

    #hyperlink
    HYPERLINKS = ("http" + "s" + OPTIONAL + ":" + FORWARD_SLASH + FORWARD_SLASH
              + NOT_SPACE + NEWLINES + ZERO_OR_MORE)
    text = re.sub(HYPERLINKS,ERASE, text)

    #hash
    text = re.sub('#',ERASE, text)

    #punctuation
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)

    #diacritics
    text = re.sub(arabic_diacritics, '', text)

    #longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    #stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)

    #emojis and emoticons
    

    return text 

In [65]:
data['tweet'] = data['tweet'].apply(preprocess)
data['tweet']

Streaming output truncated to the last 5000 lines.
ماذالوالمدارسمختلطهارتفاع التحصيل العلمي لصفوف الابتداءيه
تحكمون الوطنيه تاييد السيسي وشتم اردوغان وكلاهما حليف فتلك قسمه ضيزي
روسيا حليف للجزاءر والمغرب حليف للغرب وامريكا يقول واهم حتي الملك يحادث بوتن ارسل اشارات طمانه
وامعتصماهرب وامعتصماه انطلقتملء افواه البنات اليتملامست اسماعهم لكنهالم تلامس نخوه المعتصمسوريااراكانال
الخارجيه الايرانيه تشن هجوما لاذعا بريطانيا
اردوغان لروتانا لااستطيع تايد الانقلاب مصر والا اكون متناقض نفسي السيسي لايستطيع تاييد الثوره سوريا والا سيتناقض
يمكنك الحصول واجهه اندرويد نوجا 71 الاعلان
عبد المنعم ابو الفتوح ابعث بتهنءتي وتحياتي شباب الثوره يدافع استمرار مسيره الثوره
شدعوه رامي عياش يبيك رسمي ماتشوف شر
وزاره التخطيط تضع مشروعا لاصلاح الاقتصاد 2014 ليبدا التالي طبعا لشعبهم
اسواء هدنهفرضها كيري تشهدها تعز
اجمل ليل القاهره الساحره موسيقي راءعه كتاب شيق مصر لبنان الامارات
جزيك الله كويس حزبك مهب الريح طلع حزب يسوي وين الرواتب
روسياتقضيعليحلب المجرم بوتن وحلفاءه استماتوا بالباطل استخدموا مايملكون اموال اله 

0        محمودالعلايلياري الفريق احمدشفيق رقم مهم المعا...
1                                   فيدرر يا اجا والكبار 😍
2        الداعون لمبدا الاختلاط الجنسين كالداعين لالغاء...
3        مساكين الصبح هوما رايحين راجعين عاي غوغل تعبت ...
4        قل شرق حلب تقل حلب الشرقيه وقل غرب حلب تقل حلب...
                               ...                        
12543    هلا صاير انت يا فلعوط بدك تعطي محاضرات تويتر ب...
12544                             اله الله💜ايفونالبروفيسور
12545    اردوغان المرتكب مسلم يسمونها عمليه ارهابيه يتح...
12546                                          هاري بوتر👓🎩
12547                النت يلعن شكله طول تمام لين بدا الحفل
Name: tweet, Length: 12548, dtype: object

In [67]:
data

,tweet,sarcasm,sentiment,dialect
0,محمودالعلايلياري الفريق احمدشفيق رقم مهم المعا...,False,NEU,msa
1,فيدرر يا اجا والكبار 😍,False,NEU,msa
2,الداعون لمبدا الاختلاط الجنسين كالداعين لالغاء...,True,NEG,msa
3,مساكين الصبح هوما رايحين راجعين عاي غوغل تعبت ...,True,NEG,gulf
4,قل شرق حلب تقل حلب الشرقيه وقل غرب حلب تقل حلب...,False,NEU,msa
...,...,...,...,...
12543,هلا صاير انت يا فلعوط بدك تعطي محاضرات تويتر ب...,True,NEG,levant
12544,اله الله💜ايفونالبروفيسور,False,NEU,egypt
12545,اردوغان المرتكب مسلم يسمونها عمليه ارهابيه يتح...,False,NEU,msa
12546,هاري بوتر👓🎩,False,POS,egypt


In [68]:
data.to_csv('/content/preprocessed_1.csv')